In [1]:
import pandas as pd
from datetime import date, timedelta


## POP (Itunes) By country historic data

In [2]:
COUNTRY = "es"
DATE = date.today().strftime("%Y%m%d")
URL_ITUNES = f"https://kworb.net/pop{COUNTRY}/archive/{DATE}.html"

In [3]:
def load_popes_day(day: date) -> pd.DataFrame:
    tables = pd.read_html(URL_ITUNES)

    if not tables:
        raise ValueError(f"No tables found for {day}")

    df = tables[0].copy()

    # Normalizar columnas
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # Remove undesired columns 
    df = df.loc[:, ~df.columns.str.startswith("unnamed")]

    # Quedarnos SOLO con columnas semánticas
    cols_keep = {"pos", "artist_and_title"}
    df = df[[c for c in df.columns if c in cols_keep]]
    
    # Convert pos to numeric
    if "pos" in df.columns:
        df["pos"] = pd.to_numeric(df["pos"], errors="coerce").replace(999, pd.NA).astype("Int64")
        #Remove positions with ranking NaN
        df = df[df["pos"].notna()]

    df["date"] = pd.to_datetime(day)

    return df

In [4]:
def split_artist_title(df: pd.DataFrame) -> pd.DataFrame:
    s = df["artist_and_title"].astype(str)

    # Split SOLO en el primer " - "
    parts = s.str.split(" - ", n=1, expand=True)

    df["artist"] = parts[0].str.strip()

    if parts.shape[1] > 1:
        df["song"] = parts[1].str.strip()
    else:
        df["song"] = None

    # Remove undesired columns 
    df = df.drop(columns=["artist_and_title"])

    return df


Kworb no publica “hoy” inmediatamente. Normalmente va con 1–2 días de delay.

In [5]:
def find_latest_popes_date(max_lookback=10) -> date:
    today = date.today()

    for i in range(max_lookback):
        d = today - timedelta(days=i)
        try:
            load_popes_day(d)
            return d
        except Exception:
            continue

    raise RuntimeError("No recent Popes archive found")


In [6]:
def load_pop_history(days_back: int = 7 ) -> pd.DataFrame:
    """
    Load historical iTunes Popes data for the last `days_back` days.
    Works for anything from 1 day up to ~365 days.
    """
    last_day = find_latest_popes_date()
    days = [last_day - timedelta(days=i) for i in range(days_back)]

    dfs = []

    for d in days:
        try:
            df = load_popes_day(d)
            df = split_artist_title(df)
            dfs.append(df)
        except Exception as e:
            print(f"Skipping {d}: {e}")

    if not dfs:
        raise RuntimeError("No Popes data could be loaded")

    return pd.concat(dfs, ignore_index=True)
    

In [7]:
df_week = load_pop_history(days_back = 7)
df_week["date"].sort_values().unique()

print(f"Data for {df_week['date'].nunique()} days: from {df_week['date'].min():%Y-%m-%d} to {df_week['date'].max():%Y-%m-%d}")
df_week.columns

df_week


Data for 7 days: from 2025-12-22 to 2025-12-28


,pos,date,artist,song
0,1,2025-12-28,ROSALÍA,DESPECHÁ
1,2,2025-12-28,Bizarrap & Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52"
2,3,2025-12-28,Shakira & Rauw Alejandro,Te Felicito
3,4,2025-12-28,Manuel Turizo,La Bachata
4,5,2025-12-28,Sangiovanni & Aitana,mariposas
...,...,...,...,...
695,96,2025-12-22,Sofia Carson,Come Back Home
696,97,2025-12-22,OneRepublic,Run
697,98,2025-12-22,Sofia Carson,I Didn't Know
698,99,2025-12-22,Anitta & Maluma,El Que Espera


In [8]:
# Top 100 semanal:
# Usamos el MEJOR rank de la semana (criterio común)
df_top100_es = (
    df_week
    .groupby(["artist", "song"], as_index=False)
    .agg(
        best_rank=("pos", "min"),
        days_in_week=("date", "nunique")
    )
    .sort_values("best_rank")
    .head(100)
)

df_top100_es

,artist,song,best_rank,days_in_week
74,ROSALÍA,DESPECHÁ,1,7
13,Bizarrap & Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",2,7
90,Shakira & Rauw Alejandro,Te Felicito,3,7
59,Manuel Turizo,La Bachata,4,7
87,Sangiovanni & Aitana,mariposas,5,7
...,...,...,...,...
93,Sofia Carson,Come Back Home,96,7
68,OneRepublic,Run,97,7
94,Sofia Carson,I Didn't Know,98,7
6,Anitta & Maluma,El Que Espera,99,7


## WorldWide Ranking (all countries)

In [9]:
DATE = date.today().strftime("%Y%m%d")
print(DATE)
URL_WW = f"https://kworb.net/ww/archive/{DATE}.html"

20251228


In [10]:
from datetime import date
import pandas as pd

COUNTRY_COLS = {"us", "uk", "es"}
CORE_COLS = {"pos", "artist_and_title", "date", "days", "peak", "points", "pos_change", "pts+",	"tpts"}

def load_ww_day(day: date, country = "es") -> pd.DataFrame:
    url = f"https://kworb.net/ww/archive/{day.strftime("%Y%m%d")}.html"
    tables = pd.read_html(url)

    if not tables:
        raise ValueError(f"No tables found for {day}")

    df = tables[0].copy()

    # Normalizar columnas
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # Eliminar columnas basura
    df = df.loc[:, ~df.columns.str.startswith("unnamed")]

    # Renombrar columnas clave (WW schema)
    rename_map = {
        "pos": "pos",
        "artist_and_title": "artist_and_title",
        "days": "days",
        "pk": "peak",
        "pts": "points",
        "p+": "pos_change",
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Limpiar ranking
    if "pos" in df.columns:
        df["pos"] = (
            pd.to_numeric(df["pos"], errors="coerce")
              .replace(999, pd.NA)
              .astype("Int64")
        )
        df = df[df["pos"].notna()]

    cols_keep = [c for c in df.columns if c in CORE_COLS or c in COUNTRY_COLS]
    df = df[cols_keep]


    # Añadir fecha
    df["date"] = pd.to_datetime(day)

    df = df[df["es"].notna()]

    print(len(df))

    

    return df

load_ww_day(date.today() - timedelta(days=1))

41


,pos,pos_change,artist_and_title,days,peak,points,pts+,tpts,us,uk,es,date
0,1,+1,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI & KPop Dem...",150,1,18630,394,2.946,2.0,3.0,3.0,2025-12-27
1,2,+1,Taylor Swift - The Fate of Ophelia,79,1,17464,-80,1.501,3.0,2.0,11.0,2025-12-27
2,3,+1,Kato - Turn the Lights Off,27,2,15874,1049,0.393,5.0,4.0,6.0,2025-12-27
3,4,-3,Mariah Carey - All I Want for Christmas Is You,834,1,15084,-4094,8.977,6.0,19.0,4.0,2025-12-27
4,5,+10,Kate Bush - Running Up That Hill,261,1,11827,4291,2.162,13.0,18.0,14.0,2025-12-27
6,7,+4,RAYE - WHERE IS MY HUSBAND,100,3,11460,1632,0.937,17.0,1.0,18.0,2025-12-27
7,8,-3,Chris Rea - Driving Home for Christmas,422,2,11330,-2982,1.769,65.0,7.0,79.0,2025-12-27
8,9,-3,Wham! - Last Christmas,641,2,11013,-2975,4.459,14.0,63.0,12.0,2025-12-27
9,10,-3,Sia - Snowman,313,3,9766,-1566,1.620,40.0,25.0,17.0,2025-12-27
12,13,+3,Taylor Swift - Opalite,65,3,8529,1138,0.630,8.0,14.0,81.0,2025-12-27


## Save Data to Hopsworks
- Itunes Daily Spain raw data
- Itunes Weekly Spain feature ingeneering
- WW daily raw data
- WW weekly feature engineering

In [11]:
import pandas as pd

def normalize_text(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.lower()
         .str.replace(r"\s+", " ", regex=True)
    )
    
def prepare_itunes_daily_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Tipos
    df["date"] = pd.to_datetime(df["date"])

    # pos como entero nullable sin 1.0
    df["pos"] = (
        pd.to_numeric(df["pos"], errors="coerce")
          .replace(999, pd.NA)
          .astype("Int64")
    )

    # Limpieza básica de strings
    df["artist"] = df["artist"].astype(str).str.strip()
    df["song"] = df["song"].astype(str).str.strip()

    # Quitar filas inválidas
    df = df.dropna(subset=["date", "pos", "artist", "song"])

    # (Opcional) asegurar unicidad por PK quedándote con el mejor pos
    df = (
        df.sort_values(["date", "pos"])
          .drop_duplicates(subset=["date", "artist", "song"], keep="first")
          .reset_index(drop=True)
    )

    df["artist"] = normalize_text(df["artist"])
    df["song"] = normalize_text(df["song"])

    # Orden de columnas estable
    return df[["date", "pos", "artist", "song"]]


In [18]:
PREVIOUS_DAYS = 30
df_itunes_es_daily = prepare_itunes_daily_df(load_pop_history(PREVIOUS_DAYS))
df_itunes_es_daily


,date,pos,artist,song
0,2025-11-29,1,rosalía,despechá
1,2025-11-29,2,bizarrap & quevedo,"quevedo: bzrp music sessions, vol. 52"
2,2025-11-29,3,shakira & rauw alejandro,te felicito
3,2025-11-29,4,manuel turizo,la bachata
4,2025-11-29,5,sangiovanni & aitana,mariposas
...,...,...,...,...
2965,2025-12-28,96,sofia carson,come back home
2966,2025-12-28,97,onerepublic,run
2967,2025-12-28,98,sofia carson,i didn't know
2968,2025-12-28,99,anitta & maluma,el que espera


In [19]:
import os
import hopsworks

project = hopsworks.login(
    project=os.getenv("HOPSWORKS_PROJECT"),
    api_key_value=os.getenv("HOPSWORKS_API_KEY"),
)

fs = project.get_feature_store()

2025-12-28 17:21:39,539 INFO: Closing external client and cleaning up certificates.
2025-12-28 17:21:39,541 INFO: Connection closed.
2025-12-28 17:21:39,542 INFO: Initializing external client
2025-12-28 17:21:39,542 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-28 17:21:40,058 WARNING: UserWarning: The installed hopsworks client version 4.6.0 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-12-28 17:21:40,987 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279154


In [20]:
fg_itunes_es_daily = fs.get_or_create_feature_group(
    name="itunes_es_daily",
    version=1,
    description="Daily iTunes ES chart snapshot from Kworb (cleaned).",
    primary_key=["date", "artist", "song"],
    event_time="date",
    online_enabled=False,
)

fg_itunes_es_daily.insert(df_itunes_es_daily, write_options={"upsert": True})


Uploading Dataframe: 100.00% |█████████████████████████████| Rows 2970/2970 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: itunes_es_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279154/jobs/named/itunes_es_daily_1_offline_fg_materialization/executions


(Job('itunes_es_daily_1_offline_fg_materialization', 'SPARK'), None)

In [21]:
def build_itunes_es_weekly(df_daily: pd.DataFrame) -> pd.DataFrame:
    df = df_daily.copy()

    # Definir semana (lunes como inicio)
    df["week_start"] = df["date"].dt.to_period("W-MON").apply(lambda x: x.start_time)

    weekly = (
        df.groupby(["artist", "song", "week_start"])
          .agg(
              best_rank_week=("pos", "min"),
              avg_rank_week=("pos", "mean"),
              days_on_chart_week=("pos", "count"),
              first_rank_week=("pos", "first"),
              last_rank_week=("pos", "last"),
              rank_std_week=("pos", "std"),
          )
          .reset_index()
    )

    weekly["rank_delta_week"] = (
        weekly["first_rank_week"] - weekly["last_rank_week"]
    )

    weekly["is_new_entry_week"] = (
        weekly["days_on_chart_week"] <= 2
    ).astype(int)

    return weekly


In [22]:
df_itunes_es_weekly = build_itunes_es_weekly(df_itunes_es_daily)
df_itunes_es_weekly

,artist,song,week_start,best_rank_week,avg_rank_week,days_on_chart_week,first_rank_week,last_rank_week,rank_std_week,rank_delta_week,is_new_entry_week
0,a great big world,say something,2025-11-25,77,77.0,3,77,77,0.0,0,0
1,a great big world,say something,2025-12-02,77,77.0,7,77,77,0.0,0,0
2,a great big world,say something,2025-12-09,77,77.0,7,77,77,0.0,0,0
3,a great big world,say something,2025-12-16,77,77.0,7,77,77,0.0,0,0
4,a great big world,say something,2025-12-23,77,77.0,6,77,77,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
490,zzoilo & aitana,mon amour (remix),2025-11-25,11,11.0,3,11,11,0.0,0,0
491,zzoilo & aitana,mon amour (remix),2025-12-02,11,11.0,7,11,11,0.0,0,0
492,zzoilo & aitana,mon amour (remix),2025-12-09,11,11.0,7,11,11,0.0,0,0
493,zzoilo & aitana,mon amour (remix),2025-12-16,11,11.0,7,11,11,0.0,0,0
